In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from utils.logger import PrettyLogger
from utils.io_func import save_to_csv, save_to_pkl, save_to_pth
from utils.helper import DCMHelper
from config import SEED

In [ ]:
logger = PrettyLogger()
helper = DCMHelper()

In [ ]:
BASE_SITES = ["Site_A"]
TRAIN_YEARS = [str(year) for year in [2015, 2016, 2017]]
TEST_YEARS = [str(year) for year in [2018]]
DATA_DIR_TEMPLATE = "../preprocessing/out/{site}/"
X_PATH_TEMPLATE = os.path.join(DATA_DIR_TEMPLATE, "x-{year}.npy")
Y_PATH_TEMPLATE = os.path.join(DATA_DIR_TEMPLATE, "y-{year}.npy")
RESULT_DIR = "./out/DCM-20200101/{}/".format("_".join(BASE_SITES))
DEVICE = torch.device("cuda:0")

# Input

In [ ]:
def get_paths(path_template, sites, years):
    paths = []
    for site in sites:
        for year in years:
            paths.append(path_template.format(site=site, year=year))
    return paths


x_train = helper.input_x(get_paths(X_PATH_TEMPLATE, BASE_SITES, TRAIN_YEARS))
y_train = helper.input_y(get_paths(Y_PATH_TEMPLATE, BASE_SITES, TRAIN_YEARS))
x_test = helper.input_x(get_paths(X_PATH_TEMPLATE, BASE_SITES, TEST_YEARS))
y_test = helper.input_y(get_paths(Y_PATH_TEMPLATE, BASE_SITES, TEST_YEARS))

# Normalization

In [ ]:
scaler, x_train, x_test = helper.normalize_without_scaler(x_train, x_test)

# Training models

In [ ]:
train_dataloader = helper.make_data_loader(x_train, y_train, shuffle=True)
test_dataloader = helper.make_data_loader(x_test, y_test, shuffle=False)

net = helper.build_model()
helper.init_parameters(net)
net = nn.DataParallel(net, device_ids=[0, 1, 2, 3])
net.to(DEVICE)

loss_train_list, acc_train_list, attn_train_list = [], [], []
loss_test_list, acc_test_list, attn_test_list = [], [], []
helper.train_model(
    net, train_dataloader, test_dataloader, DEVICE, logger,
    loss_train_list, acc_train_list, attn_train_list,
    loss_test_list, acc_test_list, attn_test_list,
)

# Prediction

In [ ]:
y_train_soft_pred, y_train_hard_pred, attn_train = helper.predict(
    net, helper.make_data_loader(x_train, y_train, shuffle=False), DEVICE
)
y_test_soft_pred, y_test_hard_pred, attn_test = helper.predict(
    net, test_dataloader, DEVICE
)
acc_train = accuracy_score(y_train, y_train_hard_pred)
acc_test = accuracy_score(y_test, y_test_hard_pred)
logger.info("train acc:", acc_train, "test acc:", acc_test)

# Saving all

In [ ]:
save_to_csv(
    y_train_soft_pred, os.path.join(RESULT_DIR, "y_train_soft_pred.csv")
)
save_to_csv(
    y_test_soft_pred, os.path.join(RESULT_DIR, "y_test_soft_pred.csv")
)
save_to_csv(
    y_train_hard_pred, os.path.join(RESULT_DIR, "y_train_hard_pred.csv")
)
save_to_csv(
    y_test_hard_pred, os.path.join(RESULT_DIR, "y_test_hard_pred.csv")
)
save_to_csv(
    np.array([
        loss_train_list, loss_test_list, acc_train_list, acc_test_list
    ]).T,
    os.path.join(RESULT_DIR, "training_record.csv"),
    header=["training loss", "test loss", "training acc", "test acc"]
)
save_to_csv(
    np.array([[acc_train, acc_test]]),
    os.path.join(RESULT_DIR, "perf_abstract.csv"),
    header=["acc_train", "acc_test"]
)
save_to_pkl(scaler, os.path.join(RESULT_DIR, "scaler.pkl"))
save_to_pth(net, os.path.join(RESULT_DIR, "atbilstm.pth"))
save_to_csv(
    helper.train_time_list,
    os.path.join(RESULT_DIR, "train_time.csv"),
    header=["train_start_time", "train_end_time", "duration"]
)
save_to_csv(
    helper.test_time_list,
    os.path.join(RESULT_DIR, "test_time.csv"),
    header=["test_start_time", "test_end_time", "duration"]
)